In [2]:
import pandas as pd
merged_ipl_df = pd.read_csv('data/2022_ipl_ball_by_ball.csv')

C:\Users\manna\AppData\Local\Temp\ipykernel_22684\794233890.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_ipl_df = pd.read_csv('data/2022_ipl_ball_by_ball.csv')


In [3]:
print(merged_ipl_df['ID'].sum())

new_df = merged_ipl_df[['ID','Team1','Team2','WinningTeam']].copy()
print(new_df['ID'].sum())

23366636709
23366636709


In [4]:
grouped_df = new_df.groupby(new_df.columns.tolist(), sort=False)
# print(grouped_df['ID'].sum())

# print(type(grouped_df))
# print(type(new_df))

# for group,data in grouped_df:
#     print(group)
#     # print(data)

#     break

In [5]:
wins = {}

# Iterate over dataframe
for group,data in grouped_df:
    team1 = group[1]
    team2 = group[2]
    winner = group[3]

    if (team1,team2) in wins:
        (a,b,sum) = wins[(team1,team2)]
        if(winner == team1):
            wins[(team1,team2)] = (a+1,b,sum+1)
        else:
            wins[(team1,team2)] = (a,b+1,sum+1)
    elif (team2,team1) in wins:
        (a,b,sum) = wins[(team2,team1)]
        if(winner == team1):
            wins[(team2,team1)] = (a,b+1,sum+1)
        else:
            wins[(team2,team1)] = (a+1,b,sum+1)
    else:
        wins[(team1,team2)] = (0,0,0)
        (a,b,sum) = wins[(team1,team2)]
        if(winner == team1):
            wins[(team1,team2)] = (a+1,b,sum+1)
        else:
            wins[(team1,team2)] = (a,b+1,sum+1)

print(wins)

{('Rajasthan Royals', 'Gujarat Titans'): (0, 3, 3), ('Royal Challengers Bangalore', 'Rajasthan Royals'): (1, 2, 3), ('Royal Challengers Bangalore', 'Lucknow Super Giants'): (2, 0, 2), ('Sunrisers Hyderabad', 'Punjab Kings'): (1, 1, 2), ('Delhi Capitals', 'Mumbai Indians'): (1, 1, 2), ('Chennai Super Kings', 'Rajasthan Royals'): (0, 1, 1), ('Gujarat Titans', 'Royal Challengers Bangalore'): (1, 1, 2), ('Lucknow Super Giants', 'Kolkata Knight Riders'): (2, 0, 2), ('Sunrisers Hyderabad', 'Mumbai Indians'): (1, 0, 1), ('Delhi Capitals', 'Punjab Kings'): (2, 0, 2), ('Rajasthan Royals', 'Lucknow Super Giants'): (2, 0, 2), ('Chennai Super Kings', 'Gujarat Titans'): (0, 2, 2), ('Kolkata Knight Riders', 'Sunrisers Hyderabad'): (1, 1, 2), ('Punjab Kings', 'Royal Challengers Bangalore'): (2, 0, 2), ('Chennai Super Kings', 'Mumbai Indians'): (1, 1, 2), ('Rajasthan Royals', 'Delhi Capitals'): (1, 1, 2), ('Gujarat Titans', 'Lucknow Super Giants'): (2, 0, 2), ('Kolkata Knight Riders', 'Mumbai Indians'

In [6]:
count = 0
for i in wins:
    count=count+1

print(count)

45


In [7]:
wins_separated = {}
for (i,j) in wins.items():
    team1 = i[0]
    team2 = i[1]
    wins_separated[(team1,team2)] = j[0]
    wins_separated[(team2,team1)] = j[2]

count = 0
for i in wins_separated:
    count = count+1

print(count)

90


In [8]:
import plotly.graph_objs as go

unique_teams_list = list(set(merged_ipl_df['Team1']).union(set(merged_ipl_df['Team2'])))
# print(len(unique_teams_list)) 

# Create nodes for each team
nodes = []
for team in unique_teams_list:
    node = go.Scatter(
        x=[],
        y=[],
        text=[team],
        mode='markers+text',
        hoverinfo='none',
        marker=dict(
            symbol='circle',
            size=25,
            color='blue'
        ),
        textfont=dict(
            color='white',
            size=15
        )
    )
    nodes.append(node)

# Create edges for each head-to-head win
# Create edges for each head-to-head win
edges = []
win_counts = [wins_separated[(t1, t2)] for t1, t2 in wins_separated]
min_win_count = min(win_counts)
max_win_count = max(win_counts)
for (team1, team2), win_count in wins_separated.items():
    edge = go.Scatter(
        x=[unique_teams_list.index(team1), unique_teams_list.index(team2)],
        y=[1, -1],
        mode='lines',
        hoverinfo='none',
        line=dict(
            color='green' if win_count == max_win_count else 'red',
            width=5 * (win_count - min_win_count) / (max_win_count - min_win_count) + 1
        )
    )
    edges.append(edge)

In [9]:
# Create layout
layout = go.Layout(
    title='Head-to-Head Wins',
    xaxis=dict(
        showline=False,
        zeroline=False,
        showgrid=False,
        showticklabels=False
    ),
    yaxis=dict(
        showline=False,
        zeroline=False,
        showgrid=False,
        showticklabels=False
    ),
    hovermode='closest'
)

# Create figure
fig = go.Figure(data=edges + nodes, layout=layout)

# Show figure
fig.show()